In [1]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.7: Fast Qwen3 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4070 Ti SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.8.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [2]:
new_chat_template = \
    "{% for message in messages %}"\
        "{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<|im_end|>' + '\\n'}}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '<|im_start|>assistant\\n' }}"\
    "{% endif %}"
tokenizer.chat_template = new_chat_template

In [3]:
import random

from datasets import Dataset, load_dataset
import pandas as pd

from preprocess import get_as_messages


def load_and_preprocess():
    ds = load_dataset("Biddls/Onion_News")
    messages = ds["train"].take(800).map(get_as_messages, remove_columns=["text"])
    texts = tokenizer.apply_chat_template(messages['messages'], tokenize=False)
    
    series = pd.Series(texts, name="text")
    df = pd.DataFrame(series)
    return Dataset.from_pandas(df)
ds = load_and_preprocess()
ds[0]

{'text': '<|im_start|>user\nCome up with a funny, fake news headline.<|im_end|>\n<|im_start|>assistant\nRelaxed Marie Kondo Now Says She Perfectly Happy Living In Waist-High Sewage<|im_end|>\n'}

In [4]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 10,
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        
        output_dir='./finetunes/02-unsloth-baseline',
        save_steps=10,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/800 [00:00<?, ? examples/s]

In [5]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=12):   0%|          | 0/800 [00:00<?, ? examples/s]

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 800 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Step,Training Loss
10,5.547600
20,3.594400
30,3.510000
40,3.449600
50,3.413900
60,3.395300
70,3.260100
80,3.228100
90,3.327000
100,3.235900


Unsloth: Will smartly offload gradients to save VRAM!
